In [ ]:
pip install transformers datasets faiss-cpu

In [ ]:
import getpass
import os

HUGGINGFACE_API_KEY = getpass.getpass()
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY

In [ ]:
from langchain_community.embeddings.huggingface import HuggingFaceInferenceAPIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from pathlib import Path

In [ ]:
import json

# Example JSON format: [{"text": "document 1 text"}, {"text": "document 2 text"}, ...]

# File path to your JSON file
json_file_path = 'data.json'

# Load documents into a dataset
data = data = json.loads(Path(json_file_path).read_text())
texts = []
for json in data:
    desciption = json['description']
    poc_code = json['poc_code']
    vulnerable_code = json['vulnerable_code']
    text_to_embed = f"{desciption} {poc_code} {vulnerable_code}" # need to design prompt template
    texts.append(text_to_embed)
texts

In [ ]:
documents = [Document(page_content=text) for text in texts]
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACE_API_KEY,
    model_name="sentence-transformers/all-MiniLM-l6-v2",
)

# Generate embeddings for the document contents
doc_contents = [doc.page_content for doc in documents]
doc_embeddings = embeddings.embed_documents(doc_contents)

print(f"Type of doc_embeddings: {type(doc_embeddings)}")
print(f"Embedding for the first document: {doc_embeddings}")

print(f"Embedding for the first document: {doc_embeddings[0]}")

# print(texts[0])
vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
)

retriever = vectorstore.as_retriever()